In [1]:
#Se importan los paquetes necesarios para los cálculos.
import pandas as pd #manejo de datos
import numpy as np #parquete de matemática
import scipy.stats as sp #paquete de estadísticas
from sqlalchemy import create_engine #consultas a sql
import sqlalchemy #manejo de sql
import sys #manejo del sistema para recibir argumentos
import warnings #
from sqlalchemy import text
warnings.filterwarnings("ignore")
import pyodbc

import funciones

In [2]:
#Conexión a la base de datos
server = '10.0.0.153'
user = "testipc2"
pas = "Abc$2020"
#user = input('Ingrese el usuario: ')
#pas = input('Ingrese la contraseña: ')
try:
    #driver = 'SQL Server Native Client 11.0'
    driver = 'ODBC Driver 17 for SQL Server'
    database_connection = f'mssql://{user}:{pas}@{server}/master?driver={driver}'
    engine = create_engine(database_connection)
    connection = engine.connect()
    print('Conexión exitosa')
except:
    print('Fallo en la conexión')

Conexión exitosa


In [3]:
#aqui se ingresa el año y mes del indice que se va a calcular

anio = int(input('Ingrese el año: '))
mes = int(input('Ingrese el mes: '))

#anio = 2024
#mes = 1
dec = 0

In [4]:
#aca se generan losnombres de los archivos que tienen el indice anterior y el indice actual
excel_indantmes = funciones.excel_mesant(mes)
excel_indantanio = funciones.excel_anioant(anio, mes)
excel_indact01 = 'indprod'
excel_indact02 = '.xlsx'

excel_indant = f"{excel_indact01}_{excel_indantmes}_{excel_indantanio}{excel_indact02}"
excel_indactual = f"{excel_indact01}_{mes}_{anio}{excel_indact02}"

#excel_indactual
excel_indant

'indprod_3_2024.xlsx'

In [5]:
#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Codigos(connect):
    querycod = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    
    return pd.read_sql(querycod, connect, index_col = 'ArtCod')

codigos = Codigos(connection)
#codigos

In [6]:
#Se define una función para hallar cualquier conjunto de códigos.
#le asocia a la categoría su número en la base de datos
di = {'Art':6,'Gba':5,'Sub':4,'Gru':3,'Agr':2,'Div':1}
#le asocia a cada categoría la longitud de su código
leng = {'Div' : 2, 'Agr' : 3, 'Gru' : 4, 'Sub' : 5, 'Gba' : 7, 'Art': 9}

#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Nombres(datos, cod : str, connect = connection, dict = di):
    ''' 
    Esta función retorna un dataframe con los nombres y códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    if cod == 'Art':
        querycod = text(f"SELECT DISTINCT ArtCod, ArtNom FROM IPC2010_RN.dbo.IPC007")
    else:
        querycod = text(f'SELECT DISTINCT {cod}Cod, {cod}Nom FROM IPC2010_RN.dbo.IPCM0{dict[cod]}')

    nombres = pd.read_sql(querycod, connect, index_col = cod+'Cod')
    return pd.merge(nombres.reset_index(), datos.reset_index(), on = cod+'Cod',  how = 'inner').set_index(['RegCod',cod+'Cod'])

def CodigosCat(ind :str, cod, len = leng):
    ''' 
    Esta función retorna un dataframe con los códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    return pd.DataFrame([x[:len[ind]] for x in cod.index],columns=[ind+'Cod']).drop_duplicates().set_index(ind+'Cod')

In [7]:
#aqui se empieza la parte de la conexion con el otro server
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [8]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [9]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas01 = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful")
except Exception as e:
    print(f"Error: {e}")

Connection successful


In [10]:
boletas02 = boletas01
boletas02['codigo_articulo'] = boletas01['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
boletas02['decada'] = boletas02['decada'].replace(funciones.decadas)
boletas02['nt_tipo'] = boletas02['nt_tipo'].replace({351761:'S', 351762:'S'})
boletas02['nt_tipo'] = boletas02['nt_tipo'].fillna('N')
boletas02['cantidad_anterior'] = pd.to_numeric(boletas02['cantidad_anterior'], errors='coerce')
boletas02['cantidad_actual'] = pd.to_numeric(boletas02['cantidad_actual'], errors='coerce')
boletas02['precio_anterior'] = pd.to_numeric(boletas02['precio_anterior'], errors='coerce')
boletas02['precio_actual'] = pd.to_numeric(boletas02['precio_actual'], errors='coerce')
boletas02['decada'] = pd.to_numeric(boletas02['decada'], errors='coerce')
boletas02['region'] = boletas02['region'].astype(str)
boletas02['codigo_articulo'] = boletas02['codigo_articulo'].astype(str)
boletas02 = boletas02[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]
boletas02.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR', 'decada':'PerSem'}, inplace=True)
boletas02['RegCod'] = boletas02['RegCod'].replace({'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})

boletas02

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
0,01,053110202,34-4-2024,1.0,1.0,145.0,145.00,N,1
1,01,031220302,36-4-2024,1.0,1.0,250.0,250.00,N,1
2,01,072110201,42-4-2024,1.0,1.0,400.0,375.00,N,1
3,01,041110103,43-4-2024,1.0,1.0,1700.0,1700.00,N,1
4,01,3211051,52-4-2024,0.0,1.0,0.0,279.99,N,1
...,...,...,...,...,...,...,...,...,...
65126,08,111110101,23430-4-2024,0.0,1.0,0.0,35.00,N,3
65127,08,111110201,23430-4-2024,0.0,1.0,0.0,35.00,N,3
65128,08,111110201,23430-4-2024,0.0,1.0,0.0,35.00,N,3
65129,08,111110201,23430-4-2024,0.0,1.0,0.0,35.00,N,3


In [11]:
#impotamos los imputados
imputados = pd.read_excel("imputados.xlsx")
imputados['ArtCod'] = imputados['ArtCod'].astype(str)
imputados['RegCod'] = imputados['RegCod'].astype(str)
imputados['RegCod'] = imputados['RegCod'].replace({'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})
imputados.loc[imputados['ArtCod'].str.len() == 8, 'ArtCod'] = '0' + imputados['ArtCod']
imputados = imputados.drop(imputados.columns[0], axis=1)
imputados

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
0,01,011110102,313510,460.0,460.0,5.00,5.00,N,1
1,01,011130104,313550,161.0,166.0,3.60,3.60,N,1
2,01,011130301,313519,1.0,1.0,10.00,10.00,N,1
3,01,011230102,313506,460.0,460.0,17.00,17.00,N,1
4,01,011230103,313506,460.0,460.0,14.50,15.00,N,1
...,...,...,...,...,...,...,...,...,...
651,08,122210101,313518,1.0,1.0,139.50,156.50,N,1
652,08,122210201,313778,1.0,1.0,345.00,350.00,N,1
653,08,122210301,313518,1.0,1.0,60.00,59.00,N,1
654,08,123110103,313793,1.0,1.0,182.27,182.27,N,1


In [12]:
#boletas02 = pd.concat([boletas02, imputados])
#boletas02

In [13]:
boletas = boletas02[boletas02['ArtCod'].str.len() == 9]
boletas = pd.concat([boletas, imputados])
boletas.set_index(['RegCod', 'ArtCod', 'BolNum'], inplace=True)
boletas.sort_index(inplace=True)
boletas

UraChi  UreCan   ArtPhi   ArtPac ArtCR  PerSem
RegCod ArtCod    BolNum                                                      
01     011110101 1035-4-2024    400.0   440.0     7.65     7.65     N       1
                 1104-4-2024    400.0   400.0     7.50     7.00     N       2
                 113-4-2024     400.0   400.0     7.50     7.75     N       1
                 1175-4-2024    400.0   400.0     6.15     6.15     N       2
                 1220-4-2024    400.0   400.0     7.65     7.65     N       1
...                               ...     ...      ...      ...   ...     ...
08     125110401 16027-4-2024     1.0     1.0  2000.00  3800.00     S       1
                 16052-4-2024     1.0     1.0  3800.00  3800.00     N       3
                 16435-4-2024     1.0     1.0  2800.00  2800.00     N       1
                 16444-4-2024     1.0     1.0  4000.00  3800.00     S       1
                 16551-4-2024     1.0     1.0  2000.00  2000.00     N       3

[60059 rows x 6 columns]

In [14]:
'''boletas03 = boletas01
boletas03['codigo_articulo'] = boletas01['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
boletas03 = boletas03[['codigo_articulo', 'articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]

artimput = boletas03.rename(columns={'codigo_articulo':'ArtCod'})
artimput = artimput[artimput['ArtCod'].str.len() == 9]
#.value_counts()
artimput'''

"boletas03 = boletas01\nboletas03['codigo_articulo'] = boletas01['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones\nboletas03 = boletas03[['codigo_articulo', 'articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]\n\nartimput = boletas03.rename(columns={'codigo_articulo':'ArtCod'})\nartimput = artimput[artimput['ArtCod'].str.len() == 9]\n#.value_counts()\nartimput"

In [15]:
'''codigos01 = codigos 
codigos01 = codigos01.reset_index()
#artimput = pd.DataFrame(artimput['ArtCod'].unique(), columns=['ArtCod'])
#artimput['ArtCod'] = artimput['ArtCod'].astype(str)
artimput = codigos01[~codigos01['ArtCod'].isin(artimput['ArtCod'])]
artimput'''

"codigos01 = codigos \ncodigos01 = codigos01.reset_index()\n#artimput = pd.DataFrame(artimput['ArtCod'].unique(), columns=['ArtCod'])\n#artimput['ArtCod'] = artimput['ArtCod'].astype(str)\nartimput = codigos01[~codigos01['ArtCod'].isin(artimput['ArtCod'])]\nartimput"

In [16]:
'''boletas04 = boletas01[['codigo_articulo', 'articulo']].rename(columns={'codigo_articulo':'ArtCod'})
no_recod = boletas04[boletas04['ArtCod'].str.len() != 9]
no_recod'''

"boletas04 = boletas01[['codigo_articulo', 'articulo']].rename(columns={'codigo_articulo':'ArtCod'})\nno_recod = boletas04[boletas04['ArtCod'].str.len() != 9]\nno_recod"

In [17]:
'''prod_norecod = no_recod.groupby('ArtCod').value_counts().reset_index(name='Counts')
prod_norecod.sort_values('Counts', ascending=False, inplace=True)

#prod_norecod[prod_norecod['Counts'] == 147]'''

"prod_norecod = no_recod.groupby('ArtCod').value_counts().reset_index(name='Counts')\nprod_norecod.sort_values('Counts', ascending=False, inplace=True)\n\n#prod_norecod[prod_norecod['Counts'] == 147]"

In [18]:
'''#se guardan los nombres de los productos que no se pueden recodificar a la base anterior
norecodname01 = 'sinrecodificar'
norecodname02 = '.xlsx'
norecodname = f"{norecodname01}_{mes}_{anio}{norecodname02}"

prod_norecod.to_excel(norecodname)

norecodname'''

'#se guardan los nombres de los productos que no se pueden recodificar a la base anterior\nnorecodname01 = \'sinrecodificar\'\nnorecodname02 = \'.xlsx\'\nnorecodname = f"{norecodname01}_{mes}_{anio}{norecodname02}"\n\nprod_norecod.to_excel(norecodname)\n\nnorecodname'

In [19]:
#aqui voy a guardar los productos con sus precios para "imputarlos"
'''imputname01 = 'paraimputar'
imputname02 = '.xlsx'
inputname = f"{imputname01}_{mes}_{anio}{imputname02}"

artimput.to_excel(inputname, index=False)

inputname'''

'imputname01 = \'paraimputar\'\nimputname02 = \'.xlsx\'\ninputname = f"{imputname01}_{mes}_{anio}{imputname02}"\n\nartimput.to_excel(inputname, index=False)\n\ninputname'

In [20]:
#esta es la parte de los outliers
ignorar = False
if ignorar:
    ig = pd.read_excel(f'IGNORAR{anio}-{mes}.xlsx', dtype={f'ArtCod':str,f'RegCod':str})
    ig = ig.set_index(['RegCod','ArtCod','BolNum'])
else:
    ig = []

In [21]:
#Funcion para calcular las variaciones de cada cotización
def Variaciones(boleta, decada, outliers, ignorar = False):
    ''' 
    Esta función calcula las variaciones de la decada ingresada dado un arreglo de boletas y una decada
    
    Argumentos:
    boletas (DataFrame) : Un dataframe de las cotizaciones con las columnas RegCod, ArtCod, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem y el multiíndice (RegCod, ArtCod)
    dec (int) : La década en la que se va a calcular el índice, 

    '''
    boletas = boleta.copy()
    if decada == 0:
        semana = [1,2,3]
    elif decada == 12:
        semana = [1,2]
    elif decada == 23:
        semana =[2,3]
    else:
        semana = [decada]

    boletas = boletas.loc[boletas['PerSem'].isin(semana)]
    
    Variacion = pd.DataFrame((boletas['ArtPac']*boletas['UraChi'])/(boletas['ArtPhi']*boletas['UreCan']), columns = ['Var'])
    Variacion = Variacion.replace([np.nan,0,np.inf],1)
    Variacion.loc[boletas['ArtCR'] == 'S'] = 1.0
    if ignorar:
            Variacion.loc[Variacion.index.isin(outliers.index), 'Var'] = 1.0
    return Variacion

In [22]:
variaciones = Variaciones(boletas, dec, ig, ignorar)
variaciones

Var
RegCod ArtCod    BolNum                
01     011110101 1035-4-2024   0.909091
                 1104-4-2024   0.933333
                 113-4-2024    1.033333
                 1175-4-2024   1.000000
                 1220-4-2024   1.000000
...                                 ...
08     125110401 16027-4-2024  1.000000
                 16052-4-2024  1.000000
                 16435-4-2024  1.000000
                 16444-4-2024  1.000000
                 16551-4-2024  1.000000

[60059 rows x 1 columns]

In [23]:
def Periodos(connect):
        '''
        Llama las periodicidades de todos los artículos
        Args:
        connect (Connection) : conexión a la base de datos
        '''
        subquery = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 \
                WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
        query = text(f"SELECT ArtCod, \
                CASE \
                WHEN ArtPRC = 5 THEN  6\
                WHEN ArtPRC = 6 THEN  12\
                WHEN ArtPRC = 1 THEN  1\
                WHEN ArtPRC = 2 THEN  2\
                WHEN ArtPRC = 3 THEN  3\
                WHEN ArtPRC = 4 THEN  4\
                ELSE 1\
                END AS ArtPRC\
                FROM IPC2010_RN.dbo.IPC007 \
                WHERE ArtCod IN \
                ({subquery})\
                ORDER BY ArtCod")
        indice = 'ArtCod'
        return pd.read_sql(query, connect, index_col=indice)

In [24]:
#Función para calcular las variaciones acumuladas de los 441 artículos
def VarAcum(var,per, cod): #esta es la funcion que ellos llaman VarFlex
    ''' 
    Esta función calcula los índices relativos (variación acumulada) de los 441 artículos para todas las regiones y retorna un arreglo con índices (region, artículo). A los artículos que no fueron cotizados les asigna variación 1

    Args
    var (DataFrame) : un dataframe con las variaciones de las fuentes con multiíndice RegCod, ArtCod
    per (DataFrame) : dataframe con las periodicidades de los artículos con índice ArtCod
    cod (DataFrame) : un dataframe con los 441 códigos de los artículos
    '''

    agrupacion = var.groupby(level = ['RegCod','ArtCod']).agg(sp.gmean)
    periodicidad = agrupacion.join(per.astype(float), how = 'outer')
    raiz = pd.DataFrame(periodicidad['Var']**(1/(periodicidad['ArtPRC'])), columns = ['Var']).sort_index()
    arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
    Codigos = cod.copy()
    Codigos['ArtCod'] = Codigos.index
    codigos2 = Codigos.merge(arreglo, how = 'cross')
    codigos2 = codigos2.set_index(['RegCod','ArtCod'])
    codigos2['Var'] = raiz['Var']
    codigos2 = codigos2.replace(np.nan, 1)
    return codigos2

In [25]:
periodos = Periodos(connection)
Varac_prod = VarAcum(variaciones, periodos, codigos)
Varac_prod

,,Var
RegCod,ArtCod,
01,011110101,0.991423
02,011110101,1.012771
03,011110101,1.000744
04,011110101,1.005611
05,011110101,1.025595
...,...,...
04,125110501,1.045640
05,125110501,1.000000
06,125110501,1.000000


In [26]:
#Se agregan las variaciones del armado nacional
def FormacionNacional(indart):
    lista = ['071110101','071110102','071110103','072420201','073210101','083110101','083110202','083110301','083110302',\
                    '083110401','093210201','093310101','095110201','123110101','123110102','123110103','123110104','124110101','125110101','125110501']
    indices = indart.copy()
    for index in lista:
        try:
            indices.loc[(indart.index.get_level_values('ArtCod') == index),'Var'] = indart.loc['01', index]['Var']
        except KeyError:
            pass
    return indices

In [27]:
Varac_prod_01 = FormacionNacional(Varac_prod).sort_index() #variacion acumulada
Varac_prod_01

Var
RegCod ArtCod             
01     011110101  0.991423
       011110102  1.000000
       011110103  0.984788
       011120101  1.009753
       011120102  1.003824
...                    ...
08     125110101  1.000000
       125110201  0.941036
       125110301  1.000000
       125110401  1.000000
       125110501  1.000000

[3528 rows x 1 columns]

In [28]:
#Función para llamar la ponderación de cualquier categoría
def CatPon(cod : str, dic, connect):
    if cod == 'Art':
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCPH6 \
                    WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    else:
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCM0{dic[cod]}")
    
    query = text(f'SELECT RegCod, {cod}Cod, {cod}Pon\
        FROM  IPC2010_RN.dbo.IPCP0{dic[cod]}\
        WHERE {cod}Cod IN ({subquery}) \
        ORDER BY RegCod, {cod}Cod')              
    return pd.read_sql(query, connect, index_col=['RegCod', cod+'Cod'])

In [29]:
#Función de cálculo de índices (relativos o absolutos)
def CatInd(cod :str, Varac_prod_01, pond, len = leng, codigo = codigos):
    pond = pond.copy()
    pond.columns = ['Pon']
    index = Varac_prod_01.copy()
    index.columns = ['Ind']
    pond[cod+'Cod'] = pond.index.get_level_values(1).map(lambda x: x[:len[cod]])
    pond = pond.set_index([pond.index.get_level_values(0), pond[cod+'Cod'], pond.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    pond[cod+'Pon'] = pond.groupby(level = ['RegCod', cod+'Cod']).sum()
    pond[cod+'Pon']=pond[cod+'Pon'].replace(0,1)
    cont = pd.DataFrame(index['Ind']*pond['Pon']/pond[cod+'Pon'], columns = [cod+'Ind'])
    cont[cod+'Cod'] = cont.index.get_level_values(1).map(lambda x: x[:len[cod]])
    cont = cont.set_index([cont.index.get_level_values(0), cont[cod+'Cod'], cont.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    catind = cont.groupby(level = ['RegCod', cod+'Cod']).sum()

    if cod == 'Gba':
        arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
        cods = CodigosCat(cod, codigo)
        Codigos = cods.copy()
        Codigos[cod+'Cod'] = Codigos.index
        codigos2 = Codigos.merge(arreglo, how = 'cross')
        codigos2 = codigos2.set_index(['RegCod',cod+'Cod'])
        codigos2[cod+'Ind'] = catind[cod+'Ind']
        codigos2 = codigos2.replace(np.nan, 1)
        return codigos2
    else:
        return catind

In [30]:
CatPon('Gba', di, connection)

GbaPon
RegCod GbaCod          
00     0111101  0.63244
       0111201  0.17119
       0111202  1.33671
       0111203  0.93761
       0111301  2.61053
...                 ...
08     1251101  0.11386
       1251102  0.00713
       1251103  0.05303
       1251104  0.16740
       1251105  0.00000

[2511 rows x 1 columns]

In [31]:
pond = CatPon('Art', di, connection) #ponderaciones 
pond

ArtPon
RegCod ArtCod              
00     011110101   64.74764
       011110102   20.38296
       011110103   14.86940
       011120101   90.25644
       011120102    9.74356
...                     ...
08     125110101  100.00000
       125110201  100.00000
       125110301  100.00000
       125110401  100.00000
       125110501    0.00000

[3969 rows x 1 columns]

In [32]:
#Calculamos los índices relativos (variación) de gasto básico de las 8 regiones
Varprod02 = CatInd('Gba', Varac_prod_01, pond, leng)
Varprod02

,,GbaInd
RegCod,GbaCod,
01,0111101,0.992136
02,0111101,1.011090
03,0111101,1.001759
04,0111101,1.005084
05,0111101,1.018330
...,...,...
04,1251105,1.000000
05,1251105,0.000000
06,1251105,1.000000


In [33]:
#extraemos los indices anteriores
indant01 = pd.read_excel(excel_indant)
indant01.drop(indant01.columns[0], axis=1, inplace=True)
indant01['RegCod'] = indant01['RegCod'].astype(str)
indant01['RegCod'] = indant01['RegCod'].replace({'0':'00','1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})
indant01['GbaCod'] = indant01['GbaCod'].astype(str)
indant01['GbaCod'] = indant01['GbaCod'].apply(lambda x: '0' + x if len(x) == 6 else x)
indant01 = indant01[indant01['RegCod'] != '00']
indant01 = indant01.set_index(['RegCod', 'GbaCod'])
indant = indant01
indant

GbaInd
RegCod GbaCod             
01     0111101  184.520914
       0111201  250.204788
       0111202  243.197692
       0111203  152.144891
       0111301  173.187720
...                    ...
08     1251101  119.542285
       1251102   93.178209
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2232 rows x 1 columns]

In [34]:
#Función para extraer índices anteriores de cualquier categoría
def IndAnt(anio : int, mes : int, cod : str, conectar, dic = di, ano = False):
    if ano:
        mesant = mes
        anioant = anio - 1
    else:
        if mes == 1:
            anioant = anio -1
            mesant = 12
        else:
            anioant = anio
            mesant = mes-1

    query = text(f'SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_01_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_02_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_03_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_04_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_05_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_06_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_07_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_08_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        ORDER BY RegCod, {cod}Cod')
    return pd.read_sql(query, conectar, index_col=['RegCod',cod+'Cod'])

In [35]:
indanioant = IndAnt(anio, mes, 'Gba', connection, ano = True)
indanioant

GbaInd
RegCod GbaCod             
01     0111101  172.901799
       0111201  240.649206
       0111202  240.499437
       0111203  151.465571
       0111301  177.306556
...                    ...
08     1241101   96.475341
       1251101  120.596493
       1251102  115.187051
       1251103  113.873077
       1251104  117.294604

[2185 rows x 1 columns]

In [36]:
indprod = pd.DataFrame(indant['GbaInd']*Varprod02['GbaInd'], columns= ['GbaInd']).replace(np.nan, 0)
indprod #este seria el indice de productos actuales

GbaInd
RegCod GbaCod             
01     0111101  183.069791
       0111201  252.405548
       0111202  237.217041
       0111203  153.478091
       0111301  174.927678
...                    ...
08     1251101  119.542285
       1251102   87.684052
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2232 rows x 1 columns]

In [37]:
#Función para llamar las ponderaciones de cada región
def RegPon(connect):
    query = text(f'SELECT RegCod, RegPon \
        FROM IPC2010_RN.dbo.IPC001 WHERE RegCod != 0')            
    return pd.read_sql(query, connect, index_col= 'RegCod')

In [38]:
#Función para calcular el índice por república
def IndRep(cod :str, index, regpond, indponde):
    indices = index.copy()
    regpon = regpond.copy()
    indpon = indponde.copy()
    indreg = indices.loc[(indices.index.get_level_values(0) != '00')]
    indpond = indpon.loc[(indpon.index.get_level_values(0) != '00')]
    reppon = pd.DataFrame(indpond[cod+'Pon']*regpon['RegPon'],columns = ['Pon'])
    sumas = reppon.groupby(level =cod+ 'Cod').sum()
    regcont = pd.DataFrame(indreg[cod+'Ind']*reppon['Pon']/sumas['Pon'],columns = [cod+'Ind'])
    indrep = regcont.groupby(level = [cod+'Cod']).sum()
    indrep['RegCod'] = '00'
    indrep = indrep.set_index([indrep['RegCod'], indrep.index]).drop(columns = ['RegCod'])
    indices.loc[indices.index.get_level_values(0) == '00'] = indrep
    return pd.concat([indrep, indreg])

In [39]:
#ponderaciones de la region a nivel republica
regpon = RegPon(connection)
regpon

,RegPon
RegCod,
01,37.34790
02,5.40404
03,7.43584
04,6.53469
05,10.96698
06,18.53248
07,9.66149
08,4.11658


In [40]:
#ponderaciones de los productos a nivel republica
gbapon = CatPon('Gba', di, connection)
gbapon

GbaPon
RegCod GbaCod          
00     0111101  0.63244
       0111201  0.17119
       0111202  1.33671
       0111203  0.93761
       0111301  2.61053
...                 ...
08     1251101  0.11386
       1251102  0.00713
       1251103  0.05303
       1251104  0.16740
       1251105  0.00000

[2511 rows x 1 columns]

In [41]:
#indice de productos del mes anterior a nivel nacional
indprod_repant = IndRep('Gba', indant, regpon, gbapon)
indprod_repant

GbaInd
RegCod GbaCod             
00     0111101  164.850887
       0111201  226.598016
       0111202  335.465769
       0111203  142.566318
       0111301  189.155513
...                    ...
08     1251101  119.542285
       1251102   93.178209
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2511 rows x 1 columns]

In [42]:
#indice prodcutos a nivel republica del año anterior
indprod_anioant = IndRep('Gba', indanioant, regpon, gbapon)
indprod_anioant

GbaInd
RegCod GbaCod             
00     0111101  159.093939
       0111201  217.107424
       0111202  349.472639
       0111203  141.631965
       0111301  190.960104
...                    ...
08     1241101   96.475341
       1251101  120.596493
       1251102  115.187051
       1251103  113.873077
       1251104  117.294604

[2464 rows x 1 columns]

In [43]:
#indice productos a nivel republica actual
indprod_rep = IndRep('Gba', indprod, regpon, gbapon)
indprod_rep

GbaInd
RegCod GbaCod             
00     0111101  165.139014
       0111201  228.089436
       0111202  341.092589
       0111203  143.618991
       0111301  189.409389
...                    ...
08     1251101  119.542285
       1251102   87.684052
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2511 rows x 1 columns]

In [44]:
indprod_rep01 = indprod_rep.reset_index()
indprod_rep01

,RegCod,GbaCod,GbaInd
0,00,0111101,165.139014
1,00,0111201,228.089436
2,00,0111202,341.092589
3,00,0111203,143.618991
4,00,0111301,189.409389
...,...,...,...
2506,08,1251101,119.542285
2507,08,1251102,87.684052
2508,08,1251103,114.954121
2509,08,1251104,117.797094


In [45]:
indprod_rep01.to_excel(excel_indactual)

In [46]:
''' 
Aquí se pueden calcular los índices de cualquier categoría, sólo se debe cambiar el argumento Div (división) por el argumento Agr, Gru, Sub, Gba dependiendo del índice que se desee
'''
cat = 'Div'
inddiv = CatInd(cat, indprod_rep, gbapon)
pondiv = CatPon(cat, di, connection)
inddiv_rep = IndRep(cat, inddiv, regpon, pondiv)
inddiv_rep #indice de division a nivel republica actual

DivInd
RegCod DivCod            
00     01      287.332310
       02      131.010665
       03      120.376593
       04      129.183285
       05      131.622796
...                   ...
08     08      114.073729
       09      116.574607
       10      106.772716
       11      144.179594
       12      125.849647

[108 rows x 1 columns]

In [47]:
def IPC(index, pond):
    indices = index.copy()
    indices.columns = ['Ind']
    pon = pond.copy()
    pon.columns = ['Pon']
    return pd.DataFrame(indices['Ind']*pon['Pon']/100, columns = ['IPC']).groupby(level = ['RegCod']).sum()

In [48]:
'''esto seria el IPC actual'''
#IPC nacional actual
ipc_actual = IPC(inddiv, pondiv)
ipc_actual

,IPC
RegCod,
00,177.374917
01,145.668792
02,274.290863
03,163.460844
04,274.037584
05,159.171013
06,160.591061
07,243.039754
08,179.440265


In [49]:
#IPC nacional mes pasado
ipc_anterior = IPC(indprod_repant, gbapon)
ipc_anterior

,IPC
RegCod,
00,176.560963
01,145.384800
02,269.605468
03,162.587772
04,269.327980
05,159.281223
06,161.249750
07,240.480503
08,180.195276


In [50]:
ipc_anioant = IPC(indprod_anioant, gbapon)
ipc_anioant

,IPC
RegCod,
00,169.982491
01,141.868099
02,257.601105
03,160.221887
04,236.981229
05,155.764913
06,155.558618
07,234.721239
08,172.181638


Variaciones

In [51]:
#variacion intermensual
(ipc_actual-ipc_anterior)*100/ipc_anterior

,IPC
RegCod,
00,0.461004
01,0.195338
02,1.737871
03,0.536985
04,1.748650
05,-0.069192
06,-0.408490
07,1.064224
08,-0.418996


In [52]:
(ipc_actual-ipc_anioant)*100/ipc_anioant

,IPC
RegCod,
00,4.348934
01,2.679033
02,6.478916
03,2.021545
04,15.636831
05,2.186693
06,3.235078
07,3.543998
08,4.215680


Incidencias por division

In [53]:
def Incidencias(index, indexant, pond,ipcante):
    pon = pond.copy()
    pon.columns = ['Pon']
    indicesant = indexant.copy()
    indicesant.columns = ['Ind']
    indices = index.copy()
    indices.columns = ['Ind']
    return pd.DataFrame(pon['Pon']*(indices['Ind']-indicesant['Ind'])/ipcante['IPC'],columns= ['Incid'])

In [54]:
inddiv_repant = CatInd('Div', indprod_repant, gbapon)

In [55]:
incid_divrep = Incidencias(inddiv_rep, inddiv_repant, pondiv, ipc_anterior)
incid_divrep.loc[incid_divrep.index.get_level_values('RegCod') == '00']

Incid
RegCod DivCod          
00     01      0.322680
       02      0.000733
       03      0.001298
       04      0.010250
       05      0.000909
       06     -0.000235
       07      0.071858
       08      0.003554
       09      0.009121
       10      0.000062
       11      0.012543
       12      0.028310

In [56]:
incid_divrep.sort_values('Incid')

Incid
RegCod DivCod          
06     01     -0.544737
08     01     -0.529435
05     01     -0.198392
01     01     -0.006496
03     05     -0.004966
...                 ...
       01      0.262452
00     01      0.322680
07     01      1.000973
04     01      1.689565
02     01      1.703501

[108 rows x 1 columns]

Incidencia productos

In [57]:
incid_prodrep = Incidencias(indprod_rep, indprod_repant, gbapon, ipc_anterior)
incid_prodrep.loc[incid_prodrep.index.get_level_values('RegCod') == '00']

Incid
RegCod GbaCod           
00     0111101  0.001032
       0111201  0.001446
       0111202  0.042600
       0111203  0.005590
       0111301  0.003754
...                  ...
       1251101  0.000000
       1251102 -0.000706
       1251103  0.000000
       1251104  0.000000
       1251105  0.000000

[279 rows x 1 columns]

In [58]:
incid_prodrep.loc[incid_prodrep['Incid'].notnull()]

Incid
RegCod GbaCod           
00     0111101  0.001032
       0111201  0.001446
       0111202  0.042600
       0111203  0.005590
       0111301  0.003754
...                  ...
08     1251101  0.000000
       1251102 -0.000217
       1251103  0.000000
       1251104  0.000000
       1251105  0.000000

[2511 rows x 1 columns]

In [59]:
ipc_actualreg = ipc_actual.loc[ipc_actual.index != '00']
ipc_antreg = ipc_anterior.loc[ipc_anterior.index != '00']
ipc_anttotal = ipc_anterior.loc['00','IPC']

In [60]:
#incid_reg = 
pd.DataFrame(regpon['RegPon']*(ipc_actualreg['IPC']-ipc_antreg['IPC'])/ipc_anttotal, columns = ['Incid'])
#incid_reg

,Incid
RegCod,
01,0.060073
02,0.143407
03,0.036769
04,0.174307
05,-0.006846
06,-0.069138
07,0.140043
08,-0.017603
